# Forecasting CO2 emissions derived from animal protein consumption
#### 1. Data Preparation Notebook

"A large body of work has emerged on the environmental impacts of various diets, with most studies concluding that a diet rich in plant-based foods and with fewer animal source foods confers both improved health and environmental benefits." https://eatforum.org/content/uploads/2019/07/EAT-Lancet_Commission_Summary_Report.pdf


While some governments over the world have begun adopting plant-based policies to promote a more sustainable future, achieving this on a global scale remains a significant challenge.

The objective of this project is to analyze global and regional trends in animal protein consumption since 1961, assess how dietary consumption affects CO2 emissions, determine whether variables such as population growth and GDP influence these consumption patterns and predict global trends for the next years.

#### Questions

- What are the global and regional trends in animal protein consumption since 1961?
- How does protein consumption choices impact CO2 emissions?
- Do factors such as population growth and GDP influence a country's animal protein consumption and CO2 emissions?
- Can we forecast CO2 emissions derived from animal protein consumption?

#### Data

Final dataframe was accomplished by integrating four distinct datasets sourced from different data sources. These are:

- ##### Maddison Project 2023 version
MPD version 2023: Bolt, Jutta and Jan Luiten van Zanden (2024), "Maddison style estimates of the evolution of the world economy: A new 2023 update", Journal of Economic Surveys, 1–41. DOI: 10.1111/joes.12618”.
  
- ##### Animal protein consumption
Food and Agriculture Organization of the United Nations (2023) – with major processing by Our World in Data. “Daily supply of protein from beef – FAO” [dataset]. Food and Agriculture Organization of the United Nations, “Food Balances: Food Balances (-2013, old methodology and population)”; Food and Agriculture Organization of the United Nations, “Food Balances: Food Balances (2010-)” [original data].
  
- ##### Country classification
Crippa, M., Solazzo, E., Guizzardi, D. et al. Food systems are responsible for a third of global anthropogenic GHG emissions. Nat Food (2021). doi:10.1038/s43016-021-00225-9.
  
- ##### Green house emissions per 100g protein
Poore, J., & Nemecek, T. (2018). Reducing food’s environmental impacts through producers and consumers. Science. – processed by Our World in Data. “Greenhouse gas emissions per 100g protein” [dataset]. Poore, J., & Nemecek, T. (2018). Reducing food’s environmental impacts through producers and consumers. Science. [original data].


In [89]:
#Import necessary libraries
import pandas as pd

### Maddison Project 2023 version description
https://www.rug.nl/ggdc/historicaldevelopment/maddison/releases/maddison-project-database-2023

The Maddison Project Database provides information on comparative economic growth and income levels over the very long run. The 2023 version, by Jutta Bolt and Jan Luiten van Zanden, of this database covers 169 countries and the period up to 2022. 

Data from the year 1961 onwards will be considered for integration with the following datasets.

In [35]:
#Read csv and create a dataframe
maddison_project_path = r"C:\Users\Alba\Desktop\CodeOp project\Data\maddison-project.xlsx"
maddison_project_df = pd.read_excel(maddison_project_path)
maddison_project_df.head(5)

,countrycode,country,region,year,gdppc,pop
0,AFG,Afghanistan,South and South East Asia,1,NaN,NaN
1,AFG,Afghanistan,South and South East Asia,730,NaN,NaN
2,AFG,Afghanistan,South and South East Asia,1000,NaN,NaN
3,AFG,Afghanistan,South and South East Asia,1090,NaN,NaN
4,AFG,Afghanistan,South and South East Asia,1150,NaN,NaN


In [36]:
#Filter dataframe by year >= 1961, change column names and reset index:
maddison_project_df = (
    maddison_project_df[maddison_project_df['year'] >= 1961]
    .reset_index(drop=True)
    .rename(columns={"countrycode":"Code","country":"Country","region":"Region","year":"Year","gdppc":"Gdp_pc_$","pop":"Population Thousands"})
)

maddison_project_df.head(5)

,Code,Country,Region,Year,Gdp_pc_$,Population Thousands
0,AFG,Afghanistan,South and South East Asia,1961,1309.0,10043.0
1,AFG,Afghanistan,South and South East Asia,1962,1302.0,10267.0
2,AFG,Afghanistan,South and South East Asia,1963,1298.0,10501.0
3,AFG,Afghanistan,South and South East Asia,1964,1291.0,10744.0
4,AFG,Afghanistan,South and South East Asia,1965,1290.0,10998.0


In [37]:
maddison_project_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10478 entries, 0 to 10477
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Code                  10478 non-null  object 
 1   Country               10478 non-null  object 
 2   Region                10478 non-null  object 
 3   Year                  10478 non-null  int64  
 4   Gdp_pc_$              10162 non-null  float64
 5   Population Thousands  10478 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 491.3+ KB


In [38]:
#Check for null values in dataframe
maddison_project_df.isnull().sum()

Code                      0
Country                   0
Region                    0
Year                      0
Gdp_pc_$                316
Population Thousands      0
dtype: int64

In [39]:
maddison_project_df["Country"].nunique()

169

In [40]:
#Check regions in dataset by grouping by Region column and counting unique countries for each region
maddison_project_df.groupby(["Region"])[["Country"]].nunique()

,Country
Region,
East Asia,6
Eastern Europe,32
Latin America,26
Middle East and North Africa,20
South and South East Asia,16
Sub Saharan Africa,46
Western Europe,19
Western Offshoots,4


In [41]:
#Check what rows have NaN valyes in Gdp_pc_$ column
maddison_project_df["Gdp_isna"] = maddison_project_df["Gdp_pc_$"].isna()
maddison_nans_df = maddison_project_df[maddison_project_df["Gdp_isna"]]

maddison_nans_df

,Code,Country,Region,Year,Gdp_pc_$,Population Thousands,Gdp_isna
216,ARE,United Arab Emirates,Middle East and North Africa,1991,NaN,1928.840,True
217,ARE,United Arab Emirates,Middle East and North Africa,1992,NaN,2039.660,True
310,ARM,Armenia,Eastern Europe,1961,NaN,1943.799,True
311,ARM,Armenia,Eastern Europe,1962,NaN,2006.853,True
312,ARM,Armenia,Eastern Europe,1963,NaN,2065.618,True
...,...,...,...,...,...,...,...
9996,UZB,Uzbekistan,Eastern Europe,1975,NaN,13987.696,True
9997,UZB,Uzbekistan,Eastern Europe,1976,NaN,14404.009,True
9998,UZB,Uzbekistan,Eastern Europe,1977,NaN,14808.950,True
9999,UZB,Uzbekistan,Eastern Europe,1978,NaN,15207.113,True


In [42]:
#Convert to a list with the unique countries of the NaNs dataframe
maddison_nans_df["Country"].unique().tolist()

['United Arab Emirates',
 'Armenia',
 'Azerbaijan',
 'Belarus',
 'Czech Republic',
 'Estonia',
 'Georgia',
 'Kazakhstan',
 'Kyrgyzstan',
 'Lithuania',
 'Latvia',
 'Republic of Moldova',
 'D.P.R. of Korea',
 'Slovakia',
 'Tajikistan',
 'Turkmenistan',
 'Ukraine',
 'Uzbekistan']

Many of these countries were part of the Soviet Union, leading to gaps in GDP data for certain years in the dataset. After careful consideration, I have decided to replace the NaN values with 0 and retain these entries in the dataframe.

In [43]:
#Drop Gdp_isna column because is not needed
maddison_project_df = (
    maddison_project_df.drop(columns=["Gdp_isna"])
    .fillna(0)
) 

maddison_project_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10478 entries, 0 to 10477
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Code                  10478 non-null  object 
 1   Country               10478 non-null  object 
 2   Region                10478 non-null  object 
 3   Year                  10478 non-null  int64  
 4   Gdp_pc_$              10478 non-null  float64
 5   Population Thousands  10478 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 491.3+ KB


### Animal protein description
https://ourworldindata.org/grapher/animal-protein-consumption

Dataset found in Our World in Data website. The original data is from the Food and Agriculture Organization of the United Nations. Meat consumption is is measured as the average daily grams of protein available per person.

"Food Balance Sheet presents a comprehensive picture of the pattern of a country's food supply during a specified reference period. The food balance sheet shows for each food item [...] potentially available for human consumption. The total quantity of foodstuffs produced in a country added to the total quantity imported and adjusted to any change in stocks that may have occurred since the beginning of the reference period gives the supply available during that period. On the utilization side a distinction is made between the quantities exported, fed to livestock, used for seed, put to manufacture for food use and non-food uses, losses during storage and transportation, and food supplies available for human consumption. The per caput supply of each such food item available for human consumption is then obtained by dividing the respective quantity by the related data on the population actually partaking of it. Data on per caput food supplies are expressed in terms of quantity and - by applying appropriate food composition factors for all primary and processed products - also in terms of caloric value and protein and fat content."

In [45]:
#Read csv and create dataframe
animal_protein_path = r"C:\Users\Alba\Desktop\CodeOp project\Data\animal-protein-consumption.csv"
animal_protein_df = pd.read_csv(animal_protein_path)
animal_protein_df.head(5)

,Entity,Code,Year,Fish and seafood | 00002960 || Food available for consumption | 0674pc || grams of protein per day per capita,"Meat, poultry | 00002734 || Food available for consumption | 0674pc || grams of protein per day per capita","Meat, pig | 00002733 || Food available for consumption | 0674pc || grams of protein per day per capita","Meat, beef | 00002731 || Food available for consumption | 0674pc || grams of protein per day per capita","Meat, sheep and goat | 00002732 || Food available for consumption | 0674pc || grams of protein per day per capita","Meat, Other | 00002735 || Food available for consumption | 0674pc || grams of protein per day per capita",All egg products | 00002744 || Food available for consumption | 0674pc || grams of protein per day per capita,Milk - Excluding Butter | 00002848 || Food available for consumption | 0674pc || grams of protein per day per capita
0,Afghanistan,AFG,1961,0.010186,0.224101,NaN,2.027096,3.167975,0.366711,0.285220,6.346136
1,Afghanistan,AFG,1962,0.010193,0.234435,NaN,2.109914,3.068040,0.377134,0.305785,6.268587
2,Afghanistan,AFG,1963,0.010199,0.234585,NaN,2.131660,3.131195,0.458970,0.305980,6.813153
3,Afghanistan,AFG,1964,0.010205,0.244912,NaN,2.122574,3.224680,0.438801,0.316345,6.857546
4,Afghanistan,AFG,1965,0.010209,0.255223,NaN,2.103040,3.338321,0.469611,0.326686,7.319804


In [46]:
animal_protein_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12592 entries, 0 to 12591
Data columns (total 11 columns):
 #   Column                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                --------------  -----  
 0   Entity                                                                                                                12592 non-null  object 
 1   Code                                                                                                                  9973 non-null   object 
 2   Year                                                                                                                  12592 non-null  int64  
 3   Fish and seafood | 00002960 || Food available for consumption | 0674pc || grams of protein per day per capita         12592 non-null  float64
 4   Meat

In [47]:
#Check for null values in dataframe
animal_protein_df.isnull().sum()

Entity                                                                                                                     0
Code                                                                                                                    2619
Year                                                                                                                       0
Fish and seafood | 00002960 || Food available for consumption | 0674pc || grams of protein per day per capita              0
Meat, poultry | 00002734 || Food available for consumption | 0674pc || grams of protein per day per capita                 0
Meat, pig | 00002733 || Food available for consumption | 0674pc || grams of protein per day per capita                   328
Meat, beef | 00002731 || Food available for consumption | 0674pc || grams of protein per day per capita                    0
Meat, sheep and goat | 00002732 || Food available for consumption | 0674pc || grams of protein per day per capita          0


In [49]:
#Check what rows have NaN values in Code column
animal_protein_df["Code_isna"] = animal_protein_df["Code"].isna()
animal_nans_df = animal_protein_df[animal_protein_df["Code_isna"]]

animal_nans_df.head(5)

,Entity,Code,Year,Fish and seafood | 00002960 || Food available for consumption | 0674pc || grams of protein per day per capita,"Meat, poultry | 00002734 || Food available for consumption | 0674pc || grams of protein per day per capita","Meat, pig | 00002733 || Food available for consumption | 0674pc || grams of protein per day per capita","Meat, beef | 00002731 || Food available for consumption | 0674pc || grams of protein per day per capita","Meat, sheep and goat | 00002732 || Food available for consumption | 0674pc || grams of protein per day per capita","Meat, Other | 00002735 || Food available for consumption | 0674pc || grams of protein per day per capita",All egg products | 00002744 || Food available for consumption | 0674pc || grams of protein per day per capita,Milk - Excluding Butter | 00002848 || Food available for consumption | 0674pc || grams of protein per day per capita,Code_isna
61,Africa,NaN,1961,1.520641,0.445817,0.211815,2.742566,0.963021,0.815833,0.355072,2.450061,True
62,Africa,NaN,1962,1.523485,0.452302,0.198716,2.654351,0.937260,0.808983,0.356434,2.439476,True
63,Africa,NaN,1963,1.626542,0.455930,0.193932,2.630465,0.910605,0.802943,0.353518,2.400237,True
64,Africa,NaN,1964,1.740341,0.472992,0.201437,2.612586,0.916948,0.820719,0.364763,2.397780,True
65,Africa,NaN,1965,1.749703,0.483952,0.199726,2.623918,0.944596,0.820582,0.385412,2.534250,True


In [51]:
#Convert to a list with the unique countries of the NaNs dataframe
animal_nans_df["Entity"].unique().tolist()

['Africa',
 'Africa (FAO)',
 'Americas (FAO)',
 'Asia',
 'Asia (FAO)',
 'Belgium-Luxembourg (FAO)',
 'Caribbean (FAO)',
 'Central America (FAO)',
 'Central Asia (FAO)',
 'China (FAO)',
 'Eastern Africa (FAO)',
 'Eastern Asia (FAO)',
 'Eastern Europe (FAO)',
 'Europe',
 'Europe (FAO)',
 'European Union (27)',
 'European Union (27) (FAO)',
 'High-income countries',
 'Land Locked Developing Countries (FAO)',
 'Least Developed Countries (FAO)',
 'Low Income Food Deficit Countries (FAO)',
 'Low-income countries',
 'Lower-middle-income countries',
 'Micronesia (FAO)',
 'Middle Africa (FAO)',
 'Net Food Importing Developing Countries (FAO)',
 'North America',
 'Northern Africa (FAO)',
 'Northern America (FAO)',
 'Northern Europe (FAO)',
 'Oceania',
 'Oceania (FAO)',
 'Small Island Developing States (FAO)',
 'South America',
 'South America (FAO)',
 'South-eastern Asia (FAO)',
 'Southern Africa (FAO)',
 'Southern Asia (FAO)',
 'Southern Europe (FAO)',
 'Sudan (former)',
 'Upper-middle-income c

This are regions but there is already a column for region grouping, so I decide to drop eliminate the rows. 

In [52]:
#Drop NaNs
animal_protein_df = animal_protein_df.dropna(subset=["Code"])

In [53]:
animal_protein_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9973 entries, 0 to 12591
Data columns (total 12 columns):
 #   Column                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                --------------  -----  
 0   Entity                                                                                                                9973 non-null   object 
 1   Code                                                                                                                  9973 non-null   object 
 2   Year                                                                                                                  9973 non-null   int64  
 3   Fish and seafood | 00002960 || Food available for consumption | 0674pc || grams of protein per day per capita         9973 non-null   float64
 4   Meat, poul

Now, I'm just missing some values in the Pig Meat and Other Meat columns, which I decide to convert to zeros.

In [54]:
#Drop column Code_isna, change NaNs for 0 and change column names:
animal_protein_df = (
    animal_protein_df.drop(columns=["Code_isna"])
    .rename(columns={"Entity":"Country","Fish and seafood | 00002960 || Food available for consumption | 0674pc || grams of protein per day per capita"
                     :"Fish & Seafood","Meat, poultry | 00002734 || Food available for consumption | 0674pc || grams of protein per day per capita":
                     "Poultry Meat","Meat, pig | 00002733 || Food available for consumption | 0674pc || grams of protein per day per capita":
                     "Pig Meat","Meat, beef | 00002731 || Food available for consumption | 0674pc || grams of protein per day per capita"
                     :"Beef Meat","Meat, sheep and goat | 00002732 || Food available for consumption | 0674pc || grams of protein per day per capita":
                     "Lamb & Mutton", "All egg products | 00002744 || Food available for consumption | 0674pc || grams of protein per day per capita"
                     :"Eggs","Milk - Excluding Butter | 00002848 || Food available for consumption | 0674pc || grams of protein per day per capita":"Dairy",
                    "Meat, Other | 00002735 || Food available for consumption | 0674pc || grams of protein per day per capita":"Other Meat"})
    .fillna(0)
)

animal_protein_df.head(5)

,Country,Code,Year,Fish & Seafood,Poultry Meat,Pig Meat,Beef Meat,Lamb & Mutton,Other Meat,Eggs,Dairy
0,Afghanistan,AFG,1961,0.010186,0.224101,0.0,2.027096,3.167975,0.366711,0.285220,6.346136
1,Afghanistan,AFG,1962,0.010193,0.234435,0.0,2.109914,3.068040,0.377134,0.305785,6.268587
2,Afghanistan,AFG,1963,0.010199,0.234585,0.0,2.131660,3.131195,0.458970,0.305980,6.813153
3,Afghanistan,AFG,1964,0.010205,0.244912,0.0,2.122574,3.224680,0.438801,0.316345,6.857546
4,Afghanistan,AFG,1965,0.010209,0.255223,0.0,2.103040,3.338321,0.469611,0.326686,7.319804


In [55]:
#Create a column for the total grams of animal protein per capita per day
not_included = ["Country", "Code", "Year"]
columns_animal_protein = [col for col in animal_protein_df.columns if col not in not_included]
animal_protein_df["Total Protein (capita/day)"] = animal_protein_df[columns_animal_protein].sum(axis=1)
animal_protein_df.head(5)

,Country,Code,Year,Fish & Seafood,Poultry Meat,Pig Meat,Beef Meat,Lamb & Mutton,Other Meat,Eggs,Dairy,Total Protein (capita/day)
0,Afghanistan,AFG,1961,0.010186,0.224101,0.0,2.027096,3.167975,0.366711,0.285220,6.346136,12.427424
1,Afghanistan,AFG,1962,0.010193,0.234435,0.0,2.109914,3.068040,0.377134,0.305785,6.268587,12.374089
2,Afghanistan,AFG,1963,0.010199,0.234585,0.0,2.131660,3.131195,0.458970,0.305980,6.813153,13.085742
3,Afghanistan,AFG,1964,0.010205,0.244912,0.0,2.122574,3.224680,0.438801,0.316345,6.857546,13.215063
4,Afghanistan,AFG,1965,0.010209,0.255223,0.0,2.103040,3.338321,0.469611,0.326686,7.319804,13.822893


### Country classification description
https://edgar.jrc.ec.europa.eu/edgar_food#data_download


In [60]:
#Read csv and create dataframe
country_classification_path = r"C:\Users\Alba\Desktop\CodeOp project\Data\Country-classification.xlsx"
country_classification_df = pd.read_excel(country_classification_path)
country_classification_df.head(5)

,Country ISO code,Country name,Regional grouping,Country class
0,ABW,Aruba,Rest Central America,Developing
1,AFG,Afghanistan,India +,Developing
2,AGO,Angola,Southern_Africa,Developing
3,AIA,Anguilla,Rest Central America,Developing
4,AIR,Int. Aviation,Int. Aviation,0


In [61]:
#Drop columns and change column names:
country_classification_df = (
    country_classification_df.drop(columns=["Country name","Regional grouping"])
    .rename(columns={"Country ISO code":"Code","Country class":"Country Class"})
)

country_classification_df.head(5)

,Code,Country Class
0,ABW,Developing
1,AFG,Developing
2,AGO,Developing
3,AIA,Developing
4,AIR,0


In [62]:
country_classification_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Code           224 non-null    object
 1   Country Class  224 non-null    object
dtypes: object(2)
memory usage: 3.6+ KB


### Merging datasets

In [140]:
#Merge dataframes by code and year

merged_df = (
    pd.merge(maddison_project_df, 
             pd.merge(country_classification_df, animal_protein_df, on=["Code"], how="left"), on=["Code", "Year"], how="left")
)

merged_df.head(5)

,Code,Country_x,Region,Year,Gdp_pc_$,Population Thousands,Country Class,Country_y,Fish & Seafood,Poultry Meat,Pig Meat,Beef Meat,Lamb & Mutton,Other Meat,Eggs,Dairy,Total Protein (capita/day)
0,AFG,Afghanistan,South and South East Asia,1961,1309.0,10043.0,Developing,Afghanistan,0.010186,0.224101,0.0,2.027096,3.167975,0.366711,0.285220,6.346136,12.427424
1,AFG,Afghanistan,South and South East Asia,1962,1302.0,10267.0,Developing,Afghanistan,0.010193,0.234435,0.0,2.109914,3.068040,0.377134,0.305785,6.268587,12.374089
2,AFG,Afghanistan,South and South East Asia,1963,1298.0,10501.0,Developing,Afghanistan,0.010199,0.234585,0.0,2.131660,3.131195,0.458970,0.305980,6.813153,13.085742
3,AFG,Afghanistan,South and South East Asia,1964,1291.0,10744.0,Developing,Afghanistan,0.010205,0.244912,0.0,2.122574,3.224680,0.438801,0.316345,6.857546,13.215063
4,AFG,Afghanistan,South and South East Asia,1965,1290.0,10998.0,Developing,Afghanistan,0.010209,0.255223,0.0,2.103040,3.338321,0.469611,0.326686,7.319804,13.822893


In [141]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10478 entries, 0 to 10477
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Code                        10478 non-null  object 
 1   Country_x                   10478 non-null  object 
 2   Region                      10478 non-null  object 
 3   Year                        10478 non-null  int64  
 4   Gdp_pc_$                    10478 non-null  float64
 5   Population Thousands        10478 non-null  float64
 6   Country Class               8507 non-null   object 
 7   Country_y                   8507 non-null   object 
 8   Fish & Seafood              8507 non-null   float64
 9   Poultry Meat                8507 non-null   float64
 10  Pig Meat                    8507 non-null   float64
 11  Beef Meat                   8507 non-null   float64
 12  Lamb & Mutton               8507 non-null   float64
 13  Other Meat                  850

In [142]:
#Check for NaN values in dataframe
merged_df.isna().sum()

Code                             0
Country_x                        0
Region                           0
Year                             0
Gdp_pc_$                         0
Population Thousands             0
Country Class                 1971
Country_y                     1971
Fish & Seafood                1971
Poultry Meat                  1971
Pig Meat                      1971
Beef Meat                     1971
Lamb & Mutton                 1971
Other Meat                    1971
Eggs                          1971
Dairy                         1971
Total Protein (capita/day)    1971
dtype: int64

In [143]:
#Check what rows have NaN in Country Class column
merged_df["Class_isna"] = merged_df["Country Class"].isna()
merged_nans_df = merged_df[merged_df["Class_isna"]]

merged_nans_df

,Code,Country_x,Region,Year,Gdp_pc_$,Population Thousands,Country Class,Country_y,Fish & Seafood,Poultry Meat,Pig Meat,Beef Meat,Lamb & Mutton,Other Meat,Eggs,Dairy,Total Protein (capita/day),Class_isna
61,AFG,Afghanistan,South and South East Asia,2022,1357.987823,39251.804727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
123,AGO,Angola,Sub Saharan Africa,2022,6118.904897,25878.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
185,ALB,Albania,Eastern Europe,2022,12978.100729,2787.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
247,ARE,United Arab Emirates,Middle East and North Africa,2022,77203.670241,10099.280000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
309,ARG,Argentina,Latin America,2022,18292.317825,46736.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10290,YUG,Yugoslavia,Eastern Europe,2021,18491.681774,21027.533800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
10291,YUG,Yugoslavia,Eastern Europe,2022,19352.383894,20862.720055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
10353,ZAF,South Africa,Sub Saharan Africa,2022,11415.886895,63381.790000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
10415,ZMB,Zambia,Sub Saharan Africa,2022,3342.421325,20485.030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


I suspect that the 1971 rows lacking information in the animals dataset could be due to different country names or countries that did not exist in a certain year. I will have to give up some data. 

In [144]:
#Drop column Country_y and change column name
merged_df = (
    merged_df.drop(columns=["Country_y", "Class_isna"])
    .dropna(subset=["Country Class"])
    .rename(columns={"Country_x":"Country"})
)

merged_df.head(5)

,Code,Country,Region,Year,Gdp_pc_$,Population Thousands,Country Class,Fish & Seafood,Poultry Meat,Pig Meat,Beef Meat,Lamb & Mutton,Other Meat,Eggs,Dairy,Total Protein (capita/day)
0,AFG,Afghanistan,South and South East Asia,1961,1309.0,10043.0,Developing,0.010186,0.224101,0.0,2.027096,3.167975,0.366711,0.285220,6.346136,12.427424
1,AFG,Afghanistan,South and South East Asia,1962,1302.0,10267.0,Developing,0.010193,0.234435,0.0,2.109914,3.068040,0.377134,0.305785,6.268587,12.374089
2,AFG,Afghanistan,South and South East Asia,1963,1298.0,10501.0,Developing,0.010199,0.234585,0.0,2.131660,3.131195,0.458970,0.305980,6.813153,13.085742
3,AFG,Afghanistan,South and South East Asia,1964,1291.0,10744.0,Developing,0.010205,0.244912,0.0,2.122574,3.224680,0.438801,0.316345,6.857546,13.215063
4,AFG,Afghanistan,South and South East Asia,1965,1290.0,10998.0,Developing,0.010209,0.255223,0.0,2.103040,3.338321,0.469611,0.326686,7.319804,13.822893


In [146]:
merged_df.isnull().sum()

Code                          0
Country                       0
Region                        0
Year                          0
Gdp_pc_$                      0
Population Thousands          0
Country Class                 0
Fish & Seafood                0
Poultry Meat                  0
Pig Meat                      0
Beef Meat                     0
Lamb & Mutton                 0
Other Meat                    0
Eggs                          0
Dairy                         0
Total Protein (capita/day)    0
dtype: int64

Now I have no Null values in my dataframe so I can continue with the Emissions per Protein dataset.

### Green house emissions per 100g protein description
https://ourworldindata.org/grapher/ghg-per-protein-poore

Emissions are measured in kilograms of carbon dioxide equivalents, kgCO2eq, which takes into account not only carbon dioxide but also other greenhouse gases, such as methane and nitrous oxide, by converting them to carbon dioxide equivalents with the same global warming potential.

In [147]:
#Read csv and create dataframe
emissions_protein_path = r"C:\Users\Alba\Desktop\CodeOp project\Data\ghg-per-protein-poore.csv"
emissions_protein_df = pd.read_csv(emissions_protein_path)
emissions_protein_df.head(5)

,Entity,Code,Year,"GHG emissions per 100g protein (Poore & Nemecek, 2018)"
0,Apples,NaN,2010,14.333333
1,Bananas,NaN,2010,9.555556
2,Beef (beef herd),NaN,2010,49.889669
3,Beef (dairy herd),NaN,2010,16.869301
4,Berries & Grapes,NaN,2010,15.300000


In [148]:
#Drop column "Code" and "Year" and change column name:
emissions_protein_df = (
    emissions_protein_df.drop(columns=["Code", "Year"])
    .rename(columns={"Entity":"Food Type","GHG emissions per 100g protein (Poore & Nemecek, 2018)":"kgCO₂_p100gr",})
)
emissions_protein_df.head(5)

,Food Type,kgCO₂_p100gr
0,Apples,14.333333
1,Bananas,9.555556
2,Beef (beef herd),49.889669
3,Beef (dairy herd),16.869301
4,Berries & Grapes,15.300000


In [149]:
#Change food type name to be the same as the other df
emissions_protein_df.at[2, "Food Type"] = "Beef Meat"

#Calculate mean of Fish and Prawns
fish_prawn_mean = ((
    emissions_protein_df.loc[12, "kgCO₂_p100gr"]
    + emissions_protein_df.loc[26, "kgCO₂_p100gr"])
    /2
) 

#Create Fish and Prawns dictionary, convert to dataframe and add to emissions dataframe
fish_prawn_row = {"Food Type":"Fish & Seafood","kgCO₂_p100gr":fish_prawn_mean}
fish_prawn_df = pd.DataFrame([fish_prawn_row])
emissions_protein_df = pd.concat([emissions_protein_df, fish_prawn_df], ignore_index=True)

emissions_protein_df

,Food Type,kgCO₂_p100gr
0,Apples,14.333333
1,Bananas,9.555556
2,Beef Meat,49.889669
3,Beef (dairy herd),16.869301
4,Berries & Grapes,15.300000
5,Brassicas,4.636364
6,Cassava,14.666667
7,Cheese,10.815217
8,Citrus Fruit,6.500000
9,Coffee,35.662500


In [150]:
#Calculate mean of Milk and Cheese
milk_cheese_mean = (
    (emissions_protein_df.loc[7, "kgCO₂_p100gr"]
    + emissions_protein_df.loc[17, "kgCO₂_p100gr"])
    /2
) 

#Create Dairy dictionary, convert to dataframe and add to emissions dataframe
dairy_row = {"Food Type":"Dairy","kgCO₂_p100gr":milk_cheese_mean}
dairy_df = pd.DataFrame([dairy_row])
emissions_protein_df = pd.concat([emissions_protein_df, dairy_df], ignore_index=True)

emissions_protein_df

,Food Type,kgCO₂_p100gr
0,Apples,14.333333
1,Bananas,9.555556
2,Beef Meat,49.889669
3,Beef (dairy herd),16.869301
4,Berries & Grapes,15.300000
5,Brassicas,4.636364
6,Cassava,14.666667
7,Cheese,10.815217
8,Citrus Fruit,6.500000
9,Coffee,35.662500


In [151]:
#Calculate CO2 per 1 gram of protein instead of 100
emissions_protein_df["kgCO₂_p1gr"] = emissions_protein_df["kgCO₂_p100gr"] / 100

emissions_protein_df.head(5)

,Food Type,kgCO₂_p100gr,kgCO₂_p1gr
0,Apples,14.333333,0.143333
1,Bananas,9.555556,0.095556
2,Beef Meat,49.889669,0.498897
3,Beef (dairy herd),16.869301,0.168693
4,Berries & Grapes,15.300000,0.153000


In [152]:
#Create a dictionary from the dataframe to make calculations for the merged dataframe
emissions_todict_df = emissions_protein_df.drop(columns=["kgCO₂_p100gr"])
emissions_dict = emissions_todict_df.set_index('Food Type').T.to_dict('records')[0]

emissions_dict

{'Apples': 0.1433333333,
 'Bananas': 0.09555555556,
 'Beef Meat': 0.4988966901,
 'Beef (dairy herd)': 0.16869300909999999,
 'Berries & Grapes': 0.153,
 'Brassicas': 0.046363636359999995,
 'Cassava': 0.1466666667,
 'Cheese': 0.10815217390000001,
 'Citrus Fruit': 0.065,
 'Coffee': 0.356625,
 'Dark Chocolate': 0.9329999999999999,
 'Eggs': 0.04208723864,
 'Fish (farmed)': 0.059767594830000007,
 'Grains': 0.027000000000000003,
 'Groundnuts': 0.01233766234,
 'Lamb & Mutton': 0.1985007496,
 'Maize': 0.01789473684,
 'Milk': 0.095,
 'Nuts': 0.0026331904499999996,
 'Oatmeal': 0.01907692308,
 'Onions & Leeks': 0.03846153846,
 'Other Pulses': 0.00836057917,
 'Peas': 0.00441044104,
 'Pig Meat': 0.0760815822,
 'Potatoes': 0.027058823529999998,
 'Poultry Meat': 0.05698614319,
 'Prawns (farmed)': 0.18192281650000003,
 'Rice': 0.06267605634,
 'Root Vegetables': 0.043,
 'Tofu': 0.01975,
 'Tomatoes': 0.19,
 'Wheat & Rye': 0.01286885246,
 'Fish & Seafood': 0.120845205665,
 'Dairy': 0.10157608695}

In [153]:
#Define a function to calculate CO2 based on consumption by country and year
def calculate_CO2(df, dict):
    """Takes the grams of protein of each food type and multiplies it by the CO2 Kg of 1 gram of that food type """
    for col in df.columns:
        if col in dict:
            df[col+' CO2'] = df.apply(lambda row: row[col]*dict[col], axis=1)
    return df

merged_df = calculate_CO2(merged_df, emissions_dict)

merged_df.head(5)


,Code,Country,Region,Year,Gdp_pc_$,Population Thousands,Country Class,Fish & Seafood,Poultry Meat,Pig Meat,...,Eggs,Dairy,Total Protein (capita/day),Fish & Seafood CO2,Poultry Meat CO2,Pig Meat CO2,Beef Meat CO2,Lamb & Mutton CO2,Eggs CO2,Dairy CO2
0,AFG,Afghanistan,South and South East Asia,1961,1309.0,10043.0,Developing,0.010186,0.224101,0.0,...,0.285220,6.346136,12.427424,0.001231,0.012771,0.0,1.011312,0.628845,0.012004,0.644616
1,AFG,Afghanistan,South and South East Asia,1962,1302.0,10267.0,Developing,0.010193,0.234435,0.0,...,0.305785,6.268587,12.374089,0.001232,0.013360,0.0,1.052629,0.609008,0.012870,0.636739
2,AFG,Afghanistan,South and South East Asia,1963,1298.0,10501.0,Developing,0.010199,0.234585,0.0,...,0.305980,6.813153,13.085742,0.001233,0.013368,0.0,1.063478,0.621544,0.012878,0.692053
3,AFG,Afghanistan,South and South East Asia,1964,1291.0,10744.0,Developing,0.010205,0.244912,0.0,...,0.316345,6.857546,13.215063,0.001233,0.013957,0.0,1.058945,0.640101,0.013314,0.696563
4,AFG,Afghanistan,South and South East Asia,1965,1290.0,10998.0,Developing,0.010209,0.255223,0.0,...,0.326686,7.319804,13.822893,0.001234,0.014544,0.0,1.049200,0.662659,0.013749,0.743517


In [154]:
#Create a column for the total CO2 per capita per day
columns_CO2 = [col for col in merged_df.columns if col.endswith("CO2")]
merged_df["Total CO2 (capita/day)"] = merged_df[columns_CO2].sum(axis=1)
merged_df.head(5)

,Code,Country,Region,Year,Gdp_pc_$,Population Thousands,Country Class,Fish & Seafood,Poultry Meat,Pig Meat,...,Dairy,Total Protein (capita/day),Fish & Seafood CO2,Poultry Meat CO2,Pig Meat CO2,Beef Meat CO2,Lamb & Mutton CO2,Eggs CO2,Dairy CO2,Total CO2 (capita/day)
0,AFG,Afghanistan,South and South East Asia,1961,1309.0,10043.0,Developing,0.010186,0.224101,0.0,...,6.346136,12.427424,0.001231,0.012771,0.0,1.011312,0.628845,0.012004,0.644616,2.310778
1,AFG,Afghanistan,South and South East Asia,1962,1302.0,10267.0,Developing,0.010193,0.234435,0.0,...,6.268587,12.374089,0.001232,0.013360,0.0,1.052629,0.609008,0.012870,0.636739,2.325837
2,AFG,Afghanistan,South and South East Asia,1963,1298.0,10501.0,Developing,0.010199,0.234585,0.0,...,6.813153,13.085742,0.001233,0.013368,0.0,1.063478,0.621544,0.012878,0.692053,2.404555
3,AFG,Afghanistan,South and South East Asia,1964,1291.0,10744.0,Developing,0.010205,0.244912,0.0,...,6.857546,13.215063,0.001233,0.013957,0.0,1.058945,0.640101,0.013314,0.696563,2.424113
4,AFG,Afghanistan,South and South East Asia,1965,1290.0,10998.0,Developing,0.010209,0.255223,0.0,...,7.319804,13.822893,0.001234,0.014544,0.0,1.049200,0.662659,0.013749,0.743517,2.484903


In [155]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8507 entries, 0 to 10476
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Code                        8507 non-null   object 
 1   Country                     8507 non-null   object 
 2   Region                      8507 non-null   object 
 3   Year                        8507 non-null   int64  
 4   Gdp_pc_$                    8507 non-null   float64
 5   Population Thousands        8507 non-null   float64
 6   Country Class               8507 non-null   object 
 7   Fish & Seafood              8507 non-null   float64
 8   Poultry Meat                8507 non-null   float64
 9   Pig Meat                    8507 non-null   float64
 10  Beef Meat                   8507 non-null   float64
 11  Lamb & Mutton               8507 non-null   float64
 12  Other Meat                  8507 non-null   float64
 13  Eggs                        8507 non-

In [158]:
#Download the final dataset to continue working in another notebook
dataset_csv = merged_df.to_csv("training_dataset.csv", sep=',', index=False, encoding='utf-8')